In [20]:
def create_cypher_query(nodes, relationships, label='Taxonomy'):
    """
    Generates a Cypher query to create or merge nodes and relationships between them.
    
    This version ensures that if a node already exists, it is not recreated, but rather matched.
    
    Parameters:
    - nodes (list of tuples): Each tuple contains the label and properties for the node (e.g., ('Mammal', {'name': 'Dog'})).
    - relationships (list of tuples): Each tuple contains the nodes to create a relationship between and the relationship type (e.g., ((node1, node2), 'HAS_CHILD')).
    - label (str): The label to use for the Taxonomy nodes (default is 'Taxonomy').

    Returns:
    - str: A Cypher query string.
    """
    
    # Create node queries (using MERGE to handle existing nodes)
    node_creation_queries = []
    for node_name, properties in nodes:
        props = ', '.join([f"{key}: '{value}'" for key, value in properties.items()])
        node_creation_queries.append(f"MERGE ({node_name}:{label} {{{props}}})")
    
    # Create relationship queries
    relationship_creation_queries = []
    for (node1, node2), rel_type in relationships:
        relationship_creation_queries.append(f"MERGE ({node1})-[:{rel_type}]->({node2})")
    
    # Combine all parts into one query
    cypher_query = '\n'.join(node_creation_queries + relationship_creation_queries)

    return cypher_query

# Example Usage:

# Defining nodes and properties
nodes = [
    ('taskType_Taxon', {'name': 'TaskType', 'type':'CategoryTask'}),
    ('FullyConnectedLayerType_Taxon', {'name': 'LayerType', 'type' : 'CategoryLayer', 'LayerTypes': 'FullyConnected'}),
    ('CompositeLayerType_B_C_D_FC_Taxon', {'name': 'LayerType','type' : 'CategoryLayer', 'LayerTypes': 'BatchNorm, Concat, Dropout, FullyConnected'}),
    ('GAN_Taxon', {'name': 'GAN', 'type' : 'Network', 'task' : 'Adversarial, Classification, Generative', 'LayerTypes' : 'FullyConnected'}),
    ('AAE_Taxon', {'name': 'AAE', 'type' : 'Network', 'task' : 'Adversarial, Clustering, Discrimination, Generation, Reconstruction', 'LayerTypes' : 'BatchNorm, Concat, Dropout, FullyConnected'}),
    ('simpleClassification_Taxon', {'name': 'SimpleClassification', 'type' : 'Network', 'task' : 'SupervisedClassification', 'LayerTypes' : 'FullyConnected'}),
]

# Defining relationships
relationships = [
    
    (('taskType_Taxon', 'FullyConnectedLayerType_Taxon'), 'hasAdversarial'),
    (('taskType_Taxon', 'FullyConnectedLayerType_Taxon'), 'hasClassification'),
    (('taskType_Taxon', 'FullyConnectedLayerType_Taxon'), 'hasGenerative'),
    (('taskType_Taxon', 'FullyConnectedLayerType_Taxon'), 'hasSupervisedClassification'),
    
    (('taskType_Taxon', 'CompositeLayerType_B_C_D_FC_Taxon'), 'hasAdversarial'),
    (('taskType_Taxon', 'CompositeLayerType_B_C_D_FC_Taxon'), 'hasClustering'),
    (('taskType_Taxon', 'CompositeLayerType_B_C_D_FC_Taxon'), 'hasDiscrimination'),
    (('taskType_Taxon', 'CompositeLayerType_B_C_D_FC_Taxon'), 'hasGeneration'),
    (('taskType_Taxon', 'CompositeLayerType_B_C_D_FC_Taxon'), 'hasReconstruction'),

    (('CompositeLayerType_B_C_D_FC_Taxon', 'AAE_Taxon'), 'hasBatchNorm'),
    (('CompositeLayerType_B_C_D_FC_Taxon', 'AAE_Taxon'), 'hasConcat'),
    (('CompositeLayerType_B_C_D_FC_Taxon', 'AAE_Taxon'), 'hasDropout'),
    (('CompositeLayerType_B_C_D_FC_Taxon', 'AAE_Taxon'), 'hasFullyConnected'),
    (('FullyConnectedLayerType_Taxon', 'GAN_Taxon'), 'hasFullyConnected'),
    (('FullyConnectedLayerType_Taxon', 'simpleClassification_Taxon'), 'hasFullyConnected')
    
]

# Generating Cypher query
query = create_cypher_query(nodes, relationships)

# Print out the query
print(query)




MERGE (taskType_Taxon:Taxonomy {name: 'TaskType', type: 'CategoryTask'})
MERGE (FullyConnectedLayerType_Taxon:Taxonomy {name: 'LayerType', type: 'CategoryLayer', LayerTypes: 'FullyConnected'})
MERGE (CompositeLayerType_B_C_D_FC_Taxon:Taxonomy {name: 'LayerType', type: 'CategoryLayer', LayerTypes: 'BatchNorm, Concat, Dropout, FullyConnected'})
MERGE (GAN_Taxon:Taxonomy {name: 'GAN', type: 'Network', task: 'Adversarial, Classification, Generative', LayerTypes: 'FullyConnected'})
MERGE (AAE_Taxon:Taxonomy {name: 'AAE', type: 'Network', task: 'Adversarial, Clustering, Discrimination, Generation, Reconstruction', LayerTypes: 'BatchNorm, Concat, Dropout, FullyConnected'})
MERGE (simpleClassification_Taxon:Taxonomy {name: 'SimpleClassification', type: 'Network', task: 'SupervisedClassification', LayerTypes: 'FullyConnected'})
MERGE (taskType_Taxon)-[:hasAdversarial]->(FullyConnectedLayerType_Taxon)
MERGE (taskType_Taxon)-[:hasClassification]->(FullyConnectedLayerType_Taxon)
MERGE (taskType_Ta

In [14]:
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph

url = "bolt://localhost:7687"
username = "neo4j"
password= "neo4j"
graph = Neo4jGraph(url=url, username=username, password=password)
driver = GraphDatabase.driver(url, auth=(username, password))
session = driver.session()

In [15]:
def queryNeo4j(driver, query):
    """Runs a single Cypher query."""
    
    with driver.session() as session:
        try:
            session.run(query)
            
        except Exception as e:
            print(f"Error executing query: {e}")

In [21]:

#makes tertiary taxonomy

queryNeo4j(driver, """
MERGE (taskType_Taxon:Taxonomy {name: 'TaskType', type: 'CategoryTask'})
MERGE (FullyConnectedLayerType_Taxon:Taxonomy {name: 'LayerType', type: 'CategoryLayer', LayerTypes: 'FullyConnected'})
MERGE (CompositeLayerType_B_C_D_FC_Taxon:Taxonomy {name: 'LayerType', type: 'CategoryLayer', LayerTypes: 'BatchNorm, Concat, Dropout, FullyConnected'})
MERGE (GAN_Taxon:Taxonomy {name: 'GAN', type: 'Network', task: 'Adversarial, Classification, Generative', LayerTypes: 'FullyConnected'})
MERGE (AAE_Taxon:Taxonomy {name: 'AAE', type: 'Network', task: 'Adversarial, Clustering, Discrimination, Generation, Reconstruction', LayerTypes: 'BatchNorm, Concat, Dropout, FullyConnected'})
MERGE (simpleClassification_Taxon:Taxonomy {name: 'SimpleClassification', type: 'Network', task: 'SupervisedClassification', LayerTypes: 'FullyConnected'})
MERGE (taskType_Taxon)-[:hasAdversarial]->(FullyConnectedLayerType_Taxon)
MERGE (taskType_Taxon)-[:hasClassification]->(FullyConnectedLayerType_Taxon)
MERGE (taskType_Taxon)-[:hasGenerative]->(FullyConnectedLayerType_Taxon)
MERGE (taskType_Taxon)-[:hasSupervisedClassification]->(FullyConnectedLayerType_Taxon)
MERGE (taskType_Taxon)-[:hasAdversarial]->(CompositeLayerType_B_C_D_FC_Taxon)
MERGE (taskType_Taxon)-[:hasClustering]->(CompositeLayerType_B_C_D_FC_Taxon)
MERGE (taskType_Taxon)-[:hasDiscrimination]->(CompositeLayerType_B_C_D_FC_Taxon)
MERGE (taskType_Taxon)-[:hasGeneration]->(CompositeLayerType_B_C_D_FC_Taxon)
MERGE (taskType_Taxon)-[:hasReconstruction]->(CompositeLayerType_B_C_D_FC_Taxon)
MERGE (CompositeLayerType_B_C_D_FC_Taxon)-[:hasBatchNorm]->(AAE_Taxon)
MERGE (CompositeLayerType_B_C_D_FC_Taxon)-[:hasConcat]->(AAE_Taxon)
MERGE (CompositeLayerType_B_C_D_FC_Taxon)-[:hasDropout]->(AAE_Taxon)
MERGE (CompositeLayerType_B_C_D_FC_Taxon)-[:hasFullyConnected]->(AAE_Taxon)
MERGE (FullyConnectedLayerType_Taxon)-[:hasFullyConnected]->(GAN_Taxon)
MERGE (FullyConnectedLayerType_Taxon)-[:hasFullyConnected]->(simpleClassification_Taxon)
""")